In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
churndata=spark.read.csv("D:/Telecom Churn Data SingTel.csv", header=True,inferSchema=True) # Churn File Example

In [6]:
churndata.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account Length: integer (nullable = true)
 |-- Area Code: integer (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- International Plan: string (nullable = true)
 |-- Voice mail Plan: string (nullable = true)
 |-- Num of Voice mail Messages: integer (nullable = true)
 |-- Total Day Minutes: double (nullable = true)
 |-- Total Day Calls: integer (nullable = true)
 |-- Total day Charge: double (nullable = true)
 |-- Total Eve Minutes: double (nullable = true)
 |-- Total Eve Calls: integer (nullable = true)
 |-- Total Eve Charge: double (nullable = true)
 |-- Total Night Minutes: double (nullable = true)
 |-- Total Night Calls : integer (nullable = true)
 |-- Total Night Charge: double (nullable = true)
 |-- Total International Minutes: double (nullable = true)
 |-- Total Intl  Calls: integer (nullable = true)
 |-- Total Intl Charge: double (nullable = true)
 |-- Number Customer Service calls : integer (nullable = true)
 |-- C

In [8]:
churndata=churndata.drop('State').drop('Area Code').drop('Phone Number')

In [9]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [10]:
from pyspark.ml.feature import VectorAssembler

In [67]:
plan_indexer = StringIndexer(inputCol = 'International Plan', outputCol = 'intl_plan_indexed')
vocplan_indexer = StringIndexer(inputCol = 'Voice mail Plan', outputCol = 'voc_plan_indexed')


In [68]:
othercols=["Account Length", "Num of Voice mail Messages","Total Day Minutes", "Total Day Calls", "Total day Charge","Total Eve Minutes","Total Eve Calls","Total Eve Charge","Total Night Minutes","Total Night Calls ","Total Night Charge","Total International Minutes","Total Intl  Calls","Total Intl Charge","Number Customer Service calls "]

In [69]:
assembler = VectorAssembler(inputCols = ['intl_plan_indexed'] + ['voc_plan_indexed']+ othercols,outputCol = 'features')

In [15]:
(train, test) = churndata.randomSplit([0.7, 0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
classifier = LogisticRegression(labelCol = 'label', featuresCol = 'features')

In [19]:
label_indexer = StringIndexer(inputCol = 'Churn', outputCol = 'label')


In [20]:
pipeline = Pipeline(stages=[plan_indexer, vocplan_indexer,label_indexer, assembler, classifier])

In [21]:
model=pipeline.fit(train)

In [22]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label',metricName='f1') 

In [23]:
predictions=model.transform(train)

In [24]:
evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

0.8368201737771203

In [25]:
predictions2=model.transform(test)


In [26]:
evaluator.evaluate(predictions2, {evaluator.metricName: "f1"})

0.8420360139468045

In [27]:
evaluator1 = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label',metricName='accuracy') 
evaluator1.evaluate(predictions, {evaluator1.metricName: "accuracy"})

0.8668922729836436

In [28]:
evaluator.evaluate(predictions2, {evaluator.metricName: "accuracy"})

0.8679504814305364

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
evaluator2 = BinaryClassificationEvaluator()
evaluator2.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})


0.8208715548459191

In [31]:
evaluator2.evaluate(predictions2, {evaluator.metricName: "areaUnderROC"})

0.8436088567569529

In [56]:
### To convert yes/no, True/False to 1&0 Automatically
churndata1=spark.read.csv("F:/Telecom Churn Data SingTel.csv", header=True,inferSchema=True) # Churn File Example

In [65]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

In [71]:
classifier = RandomForestClassifier(labelCol = 'label', featuresCol = 'features')

In [72]:
pipeline = Pipeline(stages=[plan_indexer, vocplan_indexer,label_indexer, assembler, classifier])
model = pipeline.fit(train)

In [79]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [80]:
predictions2=model.transform(train)

In [82]:
evaluator=BinaryClassificationEvaluator()

In [84]:
evaluator.evaluate(predictions2,{evaluator.metricName:"areaUnderROC"})

0.9234685957442392

In [86]:
model = pipeline.fit(test)
predictions3=model.transform(test)

In [87]:
evaluator1=BinaryClassificationEvaluator()

In [88]:
evaluator1.evaluate(predictions3,{evaluator.metricName:"areaUnderROC"})

0.9648143554122234

In [90]:
### DECISION TREEE CLASSIFIER
from pyspark.ml.classification import DecisionTreeClassifier

In [102]:
dtclassifier = DecisionTreeClassifier(labelCol = 'label', featuresCol = 'features')

In [103]:
pipeline = Pipeline(stages=[plan_indexer, vocplan_indexer,label_indexer, assembler, dtclassifier])
model = pipeline.fit(train)

In [104]:
predictions4=model.transform(train)

In [105]:
evaluator4=BinaryClassificationEvaluator()
evaluator4.evaluate(predictions4,{evaluator.metricName:"areaUnderROC"})

0.4180223563236383

In [106]:
predictions5=model.transform(test)
evaluator5=BinaryClassificationEvaluator()
evaluator5.evaluate(predictions5,{evaluator.metricName:"areaUnderROC"})

0.4281973436029502